확인한 데이터를 어떤식으로 전처리를 할지
-이상치
내 목적을 이루는 딥러닝 추천모델을 설계(제작 or 찾아서 사용)
ex) CNN을 사용하거나,RNN을 사용하거나

모델의 성능 확인
(후에 모델을 선택한 이유를 발표해야함-가장 최신출시,가장 높은 정확도 같이 선택한 이유가 있어야함)
전처리를 무엇을 했는지(전처리의 전반적인 히스토리), 모델의 결과까지 설명해주면 됨


#### 가스공급량 수요예측 모델개발
Public 리더보드 : 전체 테스트 데이터 중 무작위 50%
본대회에서는 NMAE(Normalized Mean Absolute Error)-정규화 평균 절대 오차 척도를 사용하여 평가합니다.
import pandas as pd
import numpy as np


def nmae(true_df, pred_df):
    target_idx = true_df.iloc[:,0]
    pred_df = pred_df[pred_df.iloc[:,0].isin(target_idx)]
    pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True])
    true_df = true_df.sort_values(by=[true_df.columns[0]], ascending=[True])
    
    true = true_df.iloc[:,1].to_numpy()
    pred = pred_df.iloc[:,1].to_numpy()
    
    score = np.mean((np.abs(true-pred))/true)
    
    return score

#### 도배 하자 유형 분류 AI 경진대회

평가 산식 : Weighted F1 Score
Public score : 전체 테스트 데이터 중 사전 샘플링된 40% 
Private score : 전체 테스트 데이터

#### 도서 추천 알고리즘
유저 정보와 도서 정보를 바탕으로,

유저가 부여한 도서 평점을 회귀 예측하는 AI 모델을 개발

평가 산식 : RMSE (Root Mean Squared Error)
			

Public score : 전체 테스트 데이터 중 30%
Private score : 전체 테스트 데이터 중 나머지 70%

Dataset Info.

train.csv [파일]
ID : 샘플 고유 ID
User-ID : 유저 고유 ID
Book-ID : 도서 고유 ID
유저 정보
Age : 나이
Location : 지역
도서 정보
Book-Title : 도서 명
Book-Author : 도서 저자
Year-Of-Publication : 도서 출판 년도 (-1일 경우 결측 혹은 알 수 없음)
Publisher : 출판사
Book-Rating : 유저가 도서에 부여한 평점 (0점 ~ 10점)
단, 0점인 경우에는 유저가 해당 도서에 관심이 없고 관련이 없는 경우


test.csv [파일]
ID : 샘플 고유 ID
User-ID : 유저 고유 ID
Book-ID : 도서 고유 ID
유저 정보
Age : 나이
Location : 지역
도서 정보
Book-Title : 도서 명
Book-Author : 도서 저자
Year-Of-Publication : 도서 출판 년도 (-1일 경우 결측 혹은 알 수 없음)
Publisher : 출판사


sample_submission.csv [제출양식]
ID : 샘플 고유 ID
Book-Rating : 예측한 유저가 도서에 부여할 평점
단, 0점인 경우에는 유저가 해당 도서에 관심이 없고 관련이 없는 경우




In [42]:
import pandas as pd
import numpy as np

train= pd.read_csv('./book/train.csv',encoding='utf-8')
test= pd.read_csv('./book/test.csv',encoding='utf-8')
sample= pd.read_csv('./book/sample.csv',encoding='utf-8')

In [43]:
#결측치 확인
train.isnull().sum()

ID                     0
User-ID                0
Book-ID                0
Book-Rating            0
Age                    0
Location               0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64

In [44]:
train[train['Book-Rating']>0].count()

ID                     322589
User-ID                322589
Book-ID                322589
Book-Rating            322589
Age                    322589
Location               322589
Book-Title             322589
Book-Author            322589
Year-Of-Publication    322589
Publisher              322589
dtype: int64

In [45]:
# 평점이 0인 경우 유저가 해당 도서에 관심이 없고 관련이 없는 경우 이므로 
# 평점을 계산할 때 0을 포함시키지않고 계산
train= train[train['Book-Rating']>0]
train.count()

ID                     322589
User-ID                322589
Book-ID                322589
Book-Rating            322589
Age                    322589
Location               322589
Book-Title             322589
Book-Author            322589
Year-Of-Publication    322589
Publisher              322589
dtype: int64

In [46]:
# 출판연도가 -1일 경우 결측 혹은 알 수 없음 값이므로 제외
train= train[train['Year-Of-Publication']!=-1]
train.count()

ID                     317855
User-ID                317855
Book-ID                317855
Book-Rating            317855
Age                    317855
Location               317855
Book-Title             317855
Book-Author            317855
Year-Of-Publication    317855
Publisher              317855
dtype: int64

In [47]:
# 세계 최고령인 사람이 122살 까지 이므로 123살 이상은 제외
# 논리적으로 사고가 가능한 나이를 대략적으로 7세로 가정하고 그 이하는 제외
train=train[(train['Age']<122) & (train['Age']>7)]
train.count()

ID                     316771
User-ID                316771
Book-ID                316771
Book-Rating            316771
Age                    316771
Location               316771
Book-Title             316771
Book-Author            316771
Year-Of-Publication    316771
Publisher              316771
dtype: int64

In [48]:
# 2024년 이후 
train[train['Year-Of-Publication']>2024].count()

ID                     0
User-ID                0
Book-ID                0
Book-Rating            0
Age                    0
Location               0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64

In [49]:
#추가적인 전처리
# 평점을 극단적으로 높이 주거나 낮게 주는 유저가 있을수 있지만 그건 유저의 특성이라고 생각하므로 제거하지않음
#

In [50]:
def age_range(x):
    if x < 10:
        return "00대"
    elif x < 20 :
        return "10대"
    elif x < 30 :
        return "20대"
    elif x < 40 :
        return "30대"
    elif x < 50 :
        return "40대"
    elif x < 60 :
        return "50대"
    elif x < 70 :
        return "60대"
    elif x < 80 :
        return "70대"
    elif x < 90 :
        return "80대"
    elif x < 100 :
        return "90대"
    elif x < 110 :
        return "100대"
    elif x < 120 :
        return "110대"
    elif x >=120 :
        return "120대"
    
train["age_range"] = train["Age"].apply(lambda x : age_range(x))
test["age_range"] = test["Age"].apply(lambda x : age_range(x))

In [51]:
train

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher,age_range
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira,20대
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books,20대
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book,20대
5,TRAIN_000005,USER_00000,BOOK_206799,5,23.0,"sackville, new brunswick, canada",Neuromancer (Remembering Tomorrow),William Gibson,1995.0,Ace Books,20대
6,TRAIN_000006,USER_00000,BOOK_239414,9,23.0,"sackville, new brunswick, canada",The Little Prince,Antoine de Saint-ExupÃ©ry,1982.0,Harvest Books,20대
...,...,...,...,...,...,...,...,...,...,...,...
871384,TRAIN_871384,USER_92093,BOOK_268377,8,37.0,"lakewood, colorado, usa","When Hope Springs New (Canadian West, Book 4)",Janette Oke,1986.0,Bethany House,30대
871385,TRAIN_871385,USER_92093,BOOK_269242,7,37.0,"lakewood, colorado, usa",A Sand County Almanac and Sketches Here and Th...,Aldo Leopold,1989.0,Oxford University Press,30대
871386,TRAIN_871386,USER_92094,BOOK_242438,5,48.0,"ilkley, yorkshire, united kingdom",Earthdream: The Marriage of Reason and Intuition,Robert Hamilton,1991.0,Green Books,40대
871387,TRAIN_871387,USER_92095,BOOK_031650,10,35.0,"houston, texas,",A Heartbreaking Work Of Staggering Genius : A ...,Dave Eggers,2000.0,Simon &amp; Schuster,30대


In [53]:
one_hot = pd.get_dummies(train['age_range'])
train = pd.concat([train,one_hot],axis=1)
train

,ID,User-ID,Book-ID,Book-Rating,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher,...,10대,110대,20대,30대,40대,50대,60대,70대,80대,90대
0,TRAIN_000000,USER_00000,BOOK_044368,8,23.0,"sackville, new brunswick, canada",Road Taken,Rona Jaffe,2001.0,Mira,...,False,False,True,False,False,False,False,False,False,False
1,TRAIN_000001,USER_00000,BOOK_081205,8,23.0,"sackville, new brunswick, canada",Macbeth (New Penguin Shakespeare),William Shakespeare,1981.0,Penguin Books,...,False,False,True,False,False,False,False,False,False,False
4,TRAIN_000004,USER_00000,BOOK_180810,8,23.0,"sackville, new brunswick, canada",She Who Remembers,Linda Lay Shuler,1989.0,Signet Book,...,False,False,True,False,False,False,False,False,False,False
5,TRAIN_000005,USER_00000,BOOK_206799,5,23.0,"sackville, new brunswick, canada",Neuromancer (Remembering Tomorrow),William Gibson,1995.0,Ace Books,...,False,False,True,False,False,False,False,False,False,False
6,TRAIN_000006,USER_00000,BOOK_239414,9,23.0,"sackville, new brunswick, canada",The Little Prince,Antoine de Saint-ExupÃ©ry,1982.0,Harvest Books,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871384,TRAIN_871384,USER_92093,BOOK_268377,8,37.0,"lakewood, colorado, usa","When Hope Springs New (Canadian West, Book 4)",Janette Oke,1986.0,Bethany House,...,False,False,False,True,False,False,False,False,False,False
871385,TRAIN_871385,USER_92093,BOOK_269242,7,37.0,"lakewood, colorado, usa",A Sand County Almanac and Sketches Here and Th...,Aldo Leopold,1989.0,Oxford University Press,...,False,False,False,True,False,False,False,False,False,False
871386,TRAIN_871386,USER_92094,BOOK_242438,5,48.0,"ilkley, yorkshire, united kingdom",Earthdream: The Marriage of Reason and Intuition,Robert Hamilton,1991.0,Green Books,...,False,False,False,False,True,False,False,False,False,False
871387,TRAIN_871387,USER_92095,BOOK_031650,10,35.0,"houston, texas,",A Heartbreaking Work Of Staggering Genius : A ...,Dave Eggers,2000.0,Simon &amp; Schuster,...,False,False,False,True,False,False,False,False,False,False


In [ ]:
https://dacon.io/codeshare/8380?dtype=recent
https://dacon.io/competitions/official/236093/codeshare/8436?page=1&dtype=recent